In [1]:
import numpy as np, keras
#from mr import mr
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU, Dropout, Bidirectional, TimeDistributed, Masking
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

/home/soham/.env/local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
set_session(tf.Session(config=config))

In [3]:
embedding_dim = 300

In [6]:
embeddings_index = {}
f = open('../DL-NLP/Assignment 2/glove.6B.' + str(embedding_dim) + 'd.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [7]:
X_train = np.load('train_x.npy'); y_train = np.load('train_y.npy'); l_train = np.load('train_l.npy')
X_valid = np.load('valid_x.npy'); y_valid = np.load('valid_y.npy'); l_valid = np.load('valid_l.npy')
X_test  = np.load('test_x.npy' ); y_test  = np.load('test_y.npy' ); l_test  = np.load('test_l.npy' )

In [8]:
vocab_size = np.max(list(set.union(*[set(x) for x in X_train]))) + 1

In [9]:
max_length = len(X_train[0])

In [10]:
dictionary_fwd = np.load('dictionary_fwd.npy').item()

In [11]:
embedding_matrix = np.zeros((vocab_size, embedding_dim))
found = 0

for word, i in dictionary_fwd.iteritems():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        if i >= vocab_size:
            continue
        
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        found += 1

print found, 'found of', len(dictionary_fwd)

8227 found of 10098


In [13]:
# create the model
model = Sequential()
model.add(Masking(mask_value=0, input_shape=(127,)))
model.add(Embedding(vocab_size, embedding_dim, input_length=max_length, \
                    weights=[embedding_matrix], trainable=True))
model.add(Bidirectional(GRU(150, dropout=0.5, recurrent_dropout=0.5, return_sequences=True)))
model.add(Bidirectional(GRU(150, dropout=0.5, recurrent_dropout=0.5, return_sequences=True)))
model.add(Bidirectional(GRU(150, dropout=0.5, recurrent_dropout=0.5, return_sequences=True)))
model.add(TimeDistributed(Dense(3, activation='softmax')))

In [14]:
print model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_2 (Masking)          (None, 127)               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 127, 300)          3336000   
_________________________________________________________________
bidirectional_4 (Bidirection (None, 127, 300)          405900    
_________________________________________________________________
bidirectional_5 (Bidirection (None, 127, 300)          405900    
_________________________________________________________________
bidirectional_6 (Bidirection (None, 127, 300)          405900    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 127, 3)            903       
Total params: 4,554,603
Trainable params: 4,554,603
Non-trainable params: 0
_________________________________________________________________


In [15]:
from keras.callbacks import ModelCheckpoint

In [17]:
mcp = ModelCheckpoint('bidir_rnn_word2vec.hdf5', monitor="val_acc", save_best_only=True, save_weights_only=False)

In [22]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [23]:
y_train2 = np.array([to_categorical(y, 3) for y in y_train])
y_valid2 = np.array([to_categorical(y, 3) for y in y_valid])
y_test2 = np.array([to_categorical(y, 3) for y in y_test])

In [24]:
model.fit(X_train, y_train2, epochs=30, batch_size=128, validation_data=(X_valid, y_valid2), callbacks=[mcp])

Train on 2558 samples, validate on 365 samples
Epoch 1/30
2558/2558 [==============================] - 24s - loss: 0.0833 - acc: 0.9799 - val_loss: 0.0690 - val_acc: 0.9830
Epoch 2/30
2558/2558 [==============================] - 23s - loss: 0.0505 - acc: 0.9837 - val_loss: 0.0377 - val_acc: 0.9874
Epoch 3/30
2558/2558 [==============================] - 23s - loss: 0.0417 - acc: 0.9859 - val_loss: 0.0355 - val_acc: 0.9873
Epoch 4/30
2558/2558 [==============================] - 22s - loss: 0.0384 - acc: 0.9867 - val_loss: 0.0331 - val_acc: 0.9883
Epoch 5/30
2558/2558 [==============================] - 23s - loss: 0.0324 - acc: 0.9883 - val_loss: 0.0321 - val_acc: 0.9890
Epoch 6/30
2558/2558 [==============================] - 22s - loss: 0.0302 - acc: 0.9889 - val_loss: 0.0351 - val_acc: 0.9892
Epoch 7/30
2558/2558 [==============================] - 22s - loss: 0.0273 - acc: 0.9898 - val_loss: 0.0330 - val_acc: 0.9876
Epoch 8/30
2558/2558 [==============================] - 22s - loss: 0.0

In [25]:
model.load_weights('bidir_rnn_word2vec.hdf5')

In [26]:
result = model.evaluate(X_test, y_test2, batch_size=128)

732/732 [==============================] - 1s     


In [27]:
predictions = model.predict(X_test)

In [28]:
with open('predictions_rnn_word2vec.txt', 'w') as f:
    for tokens, prediction in zip(X_test, predictions):
        started = False

        for tok, pred in zip(tokens, prediction):
            if not started and tok == 1:
                started = True
            elif started and tok == 2:
                f.write('\n')
                break
            elif started:
                if np.argmax(pred) == 0:
                    f.write('O\n')
                elif np.argmax(pred) == 1:
                    f.write('T\n')
                elif np.argmax(pred) == 2:
                    f.write('D\n')